1. Данные выставления видеоиграм различных возрастных рейтингов. Представлены различные критерии, такие как: "Присутсвие алкоголя", "Присутсвие крови", "Присутсвие насилия" и т. д. Датасет взят с сайта www.kaggle.com, вот ссылка на сам датасет:
https://www.kaggle.com/datasets/imohtn/video-games-rating-by-esrb
Данные реальны. Датасет заране разделён на 2 части: Video_games_esrb_rating - обучающая и test_esrb - отложенная.

3. Виды рейтингов:
- RP - Рейтинг ожидается
- EC - Для детей младшего возраста
- E - Для всех
- ET - Для всех от 10 лет и старше
- T - Подросткам
- M - Для взрослых
- A - Только для взрослых

3. Представлены числовые и строковые типы данных.
- title - название видеогры - (строковый)
- ESRB_rating - возврастной рейтинг - (строковый)
- Остальные типы данных (32 столбца) сведетельствуют о наличии тех или иных критериев и принимают значения 0 и 1 - (числовой)

In [269]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

#библиотеки для вывода дерева
from io import StringIO
import pydotplus
from ipywidgets import Image
from sklearn.tree import export_graphviz

#Метрики оценки качества
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, RocCurveDisplay

df = pd.read_csv('Video_games_esrb_rating.csv')
df

,title,console,alcohol_reference,animated_blood,blood,blood_and_gore,cartoon_violence,crude_humor,drug_reference,fantasy_violence,...,sexual_content,sexual_themes,simulated_gambling,strong_janguage,strong_sexual_content,suggestive_themes,use_of_alcohol,use_of_drugs_and_alcohol,violence,esrb_rating
0,Monster Jam Steel Titans 2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
1,Subnautica: Below Zero,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ET
2,NIER REPLICANT VER.1.22474487139…,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,M
3,Jamestown+,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,ET
4,Neptunia Virtual Stars,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,SENRAN KAGURA Peach Beach Splash,0,0,0,0,0,0,0,0,1,...,1,1,0,1,0,0,0,0,0,M
1891,Sneaky Bears,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,T
1892,SPARC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
1893,Still Time,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,T


Проверяем на пустые ячейки

In [220]:
df.isna().sum()

title                       0
console                     0
alcohol_reference           0
animated_blood              0
blood                       0
blood_and_gore              0
cartoon_violence            0
crude_humor                 0
drug_reference              0
fantasy_violence            0
intense_violence            0
language                    0
lyrics                      0
mature_humor                0
mild_blood                  0
mild_cartoon_violence       0
mild_fantasy_violence       0
mild_language               0
mild_lyrics                 0
mild_suggestive_themes      0
mild_violence               0
no_descriptors              0
nudity                      0
partial_nudity              0
sexual_content              0
sexual_themes               0
simulated_gambling          0
strong_janguage             0
strong_sexual_content       0
suggestive_themes           0
use_of_alcohol              0
use_of_drugs_and_alcohol    0
violence                    0
esrb_ratin

Проверяем дубликаты

In [222]:
df.duplicated().sum()

0

In [223]:
ratingMapping = {'RP': 0, 'EC': 1, 'E': 2, 'ET': 3, 'T': 4, 'M': 5, 'A': 6}
df['esrb_rating'] = df['esrb_rating'].map(ratingMapping)
df

,title,console,alcohol_reference,animated_blood,blood,blood_and_gore,cartoon_violence,crude_humor,drug_reference,fantasy_violence,...,sexual_content,sexual_themes,simulated_gambling,strong_janguage,strong_sexual_content,suggestive_themes,use_of_alcohol,use_of_drugs_and_alcohol,violence,esrb_rating
0,Monster Jam Steel Titans 2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,Subnautica: Below Zero,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,NIER REPLICANT VER.1.22474487139…,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,5
3,Jamestown+,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,3
4,Neptunia Virtual Stars,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,SENRAN KAGURA Peach Beach Splash,0,0,0,0,0,0,0,0,1,...,1,1,0,1,0,0,0,0,0,5
1891,Sneaky Bears,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,4
1892,SPARC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1893,Still Time,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


Построим дерево решений

In [225]:
df_x = df.drop(columns=["esrb_rating","title"], axis=1)
df_y = df["esrb_rating"]

df_test = pd.read_csv("test_esrb.csv")
ratingMapping = {'RP': 0, 'EC': 1, 'E': 2, 'ET': 3, 'T': 4, 'M': 5, 'A': 6}
df_test['esrb_rating'] = df_test['esrb_rating'].map(ratingMapping)
df_test_x = df_test.drop(columns=["esrb_rating", "title"], axis=1)
df_test_y = df_test["esrb_rating"]

In [226]:
tree = DecisionTreeClassifier(criterion="entropy")
tree.fit(df_x, df_y)

DecisionTreeClassifier(criterion='entropy')

In [227]:
tree.score(df_test_x, df_test_y)

0.846

In [228]:
dot_data = StringIO()
export_graphviz(tree, feature_names=df_x.columns, out_file=dot_data, filled=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(value=graph.create_png())

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00@\xf1\x00\x00\x10\x07\x08\x06\x00\x00\x00h\x96.,\x00\…

Дерево получилось слишком переусложнённым. Поэтому настроим максимальную глубину дерева.

In [230]:
treeParams = {"max_depth": np.arange(1, 11), "max_features": [0.5, 0.7, 1]}
treeGrid = GridSearchCV(tree, treeParams, cv=5, n_jobs=-1)
treeGrid.fit(df_x, df_y)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(criterion='entropy'),
             n_jobs=-1,
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'max_features': [0.5, 0.7, 1]})

In [231]:
treeGrid.best_score_, treeGrid.best_params_

(0.8015831134564644, {'max_depth': 10, 'max_features': 0.7})

In [232]:
dot_data = StringIO()
export_graphviz(treeGrid.best_estimator_, feature_names=df_x.columns, out_file=dot_data, filled=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(value=graph.create_png())

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00&\x19\x00\x00\x05\xbc\x08\x02\x00\x00\x00>K\xf8Z\x00\…

Построим KNN

In [234]:
knn = KNeighborsClassifier()
knn.fit(df_x, df_y)

KNeighborsClassifier()

In [235]:
knn.score(df_test_x, df_test_y)

0.776

In [236]:
knnParams = {"n_neighbors": [1, 2, 3, 4] + list(range(5, 30, 5))}
knnGrid = GridSearchCV(knn, knnParams, cv=5)
knnGrid.fit(df_x, df_y)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 10, 15, 20, 25]})

In [237]:
knnGrid.best_score_, knnGrid.best_params_

(0.8137203166226913, {'n_neighbors': 5})

In [238]:
treePred = treeGrid.predict(df_test_x)
knnPred = knnGrid.predict(df_test_x)

Accuracy

In [240]:
accuracy_score(df_test_y, treePred)

0.76

In [241]:
accuracy_score(df_test_y, knnPred)

0.776

Precision

In [243]:
precision_score(df_test_y, treePred, average='micro')

0.76

In [244]:
precision_score(df_test_y, knnPred, average='micro')

0.776

Recall

In [246]:
recall_score(df_test_y, treePred, average='micro')

0.76

In [247]:
recall_score(df_test_y, knnPred, average='micro')

0.776

F-мера

In [249]:
f1_score(df_test_y, treePred, average='micro')

0.76

In [250]:
f1_score(df_test_y, knnPred, average='micro')

0.776

ROC_AUC

In [258]:
treeProba = treeGrid.predict_proba(df_test_x)
roc_auc_score(df_test_y, treeProba, multi_class='ovr')

0.9017666161115812

In [260]:
knnProba = knnGrid.predict_proba(df_test_x)
roc_auc_score(df_test_y, knnProba, multi_class='ovr')

0.9199747987346704